`Dataframe` to `DocumentArray`...

In [5]:
from docarray import Document, DocumentArray
import sys

sys.path.append('C:/Users/lachy/Videos/LACHY/SRI Repo/bias-project-ML')
from data_preprocess import data_selection

train_df, test_df = data_selection('C:/Users/lachy/Pictures/utkface/utkface/')

train_da = DocumentArray.from_dataframe(train_df)
test_da = DocumentArray.from_dataframe(test_df)

# for item in train_da:
#     item.summary()


C:\Users/lachy/Videos/LACHY/SRI Repo/bias-project-ML\data_preprocess.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'][i]= gender_mapper[df['gender'][i]]
C:\Users/lachy/Videos/LACHY/SRI Repo/bias-project-ML\data_preprocess.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'][i]= race_mapper[df['race'][i]]


Initializing `CLIP`...

In [6]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

`CLIP` Fine-tuning...

In [7]:
import finetuner

finetuner.login()

`Race ` and ` Gender` Fine-tuning...

In [8]:
gender_pairs = DocumentArray() # initialize a DocumentArray as final training data.

prompt = 'This is a person of '
for doc in train_da:
    pair = Document()
    doc.uri = doc.tags['filepath']
    img_chunk = doc.load_uri_to_image_tensor(224, 224)
    img_chunk.modality = 'image'
    txt_chunk = Document(content=prompt + doc.tags['gender'] + ' gender and ' + doc.tags['race'] + ' race.' )
    txt_chunk.modality = 'text'
    pair.chunks.extend([img_chunk, txt_chunk])
    # add pair to pairs
    gender_pairs.append(pair)

# Lets see the first item of the pairs
gender_pairs[0]

📄 Document: c1a2d528259f7f9e7b832045f1e996ae
└── 💠 Chunks
    ├── 📄 Document: 8033cda3e333ea306d1d7bd96e017ebc
    │   ╭───────────┬──────────────────────────────────────────────────────────────────╮
    │   │ Attribute │ Value                                                            │
    │   ├───────────┼──────────────────────────────────────────────────────────────────┤
    │   │ tensor    │ <class 'numpy.ndarray'> in shape (224, 224, 3), dtype: uint8     │
    │   │ mime_type │ image/jpeg                                                       │
    │   │ uri       │ C:/Users/lachy/Pictures/utkface/utkface/100_0_0_201701122152403… │
    │   │ tags      │ {'filepath':                                                     │
    │   │           │ 'C:/Users/lachy/Pictures/utkface/utkface/100_0_0_20170112215240… │
    │   │           │ 'race': 'white', 'gender': 'male', 'age': 100, 'filename':       │
    │   │           │ '100_0_0_20170112215240346.jpg.chip.jpg'}                        │
    │   │ modality  │ image                                                            │
    │   ╰───────────┴──────────────────────────────────────────────────────────────────╯
    └── 📄 Document: 111fd89056dff9bb8c10a45e20842446
        ╭───────────────┬──────────────────────────────────────────────────────────────╮
        │ Attribute     │ Value                                                        │
        ├───────────────┼──────────────────────────────────────────────────────────────┤
        │ text          │ This is a person of male gender and white race.              │
        │ modality      │ text                                                         │
        ╰───────────────┴──────────────────────────────────────────────────────────────╯

In [44]:
len(gender_pairs)

18966

Training `CLIP` model...

In [46]:
run = finetuner.fit(
    model='openai/clip-vit-base-patch32', # fine-tune CLIP
    train_data=gender_pairs,   
    learning_rate=1e-5,
    loss='CLIPLoss'
)

Pushing a DocumentArray to Hubble under the name finetuner-dastorage-default-elated-easley-train ...


Output()

In [47]:
for entry in run.stream_logs():
    print(entry)

Output()

[05:13:12] INFO     Starting finetuner training run ...                                                  __main__.py:350
DEBUG    Found Jina AI Cloud authentication token                                             __main__.py:362
DEBUG    Running in online mode                                                               __main__.py:363
INFO     Reading config ...                                                                   __main__.py:370
DEBUG    Reading config from stream                                                           __main__.py:382
INFO     Parsing config ...                                                                   __main__.py:385
INFO     Config loaded 📜                                                                     __main__.py:389
INFO     Run name: elated-easley                                                              __main__.py:391
INFO     Experiment name: default                                                             __main__.py:392


Saving the artifact

In [2]:
artifact = run.save_artifact('/artifact/')

NameError: name 'run' is not defined

Saving encoders...

In [9]:
clip_text_encoder = finetuner.get_model(artifact='/artifact/elated-easley.zip', select_model='clip-text')
clip_image_encoder = finetuner.get_model(artifact='/artifact/elated-easley.zip', select_model='clip-vision')

Saving images embeddings...

In [10]:
for d in test_da:
    d.uri = d.tags['filepath']
images = finetuner.encode(model=clip_image_encoder, data=test_da)

Output()

Create query texts to evaluate the model...

In [39]:
race_labels = ['black', 'white', 'asian', 'indian', 'other']
race_tks = []
for label in race_labels:
    race_tks.append('A photo of a person of ' + label + ' race.')

sex_labels = ['male', 'female']
sex_tks = []
for label in sex_labels:
    sex_tks.append('A photo of a person of ' + label + ' gender.')

sex_tks
race_tks


['A photo of a person of black race.',
 'A photo of a person of white race.',
 'A photo of a person of asian race.',
 'A photo of a person of indian race.',
 'A photo of a person of other race.']

Saving texts embeddings...(sex)

In [31]:
text_docs = DocumentArray()
for item in sex_tks:
    text_docs.append(Document(content=item))
text_docs[0].summary()
text_docs[1].summary()

finetuner.encode(model=clip_text_encoder, data=text_docs)

📄 Document: 2869bb3a91c69e657e5a455ab69e7f85
╭──────────────────┬───────────────────────────────────────────────────────────╮
│ Attribute        │ Value                                                     │
├──────────────────┼───────────────────────────────────────────────────────────┤
│ text             │ A photo of a person of male gender.                       │
╰──────────────────┴───────────────────────────────────────────────────────────╯

📄 Document: e35c5ed55951f6c736855b2ccc33d273
╭─────────────────┬────────────────────────────────────────────────────────────╮
│ Attribute       │ Value                                                      │
├─────────────────┼────────────────────────────────────────────────────────────┤
│ text            │ A photo of a person of female gender.                      │
╰─────────────────┴────────────────────────────────────────────────────────────╯

Output()

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Type                   DocumentArrayInMemory         │
│   Length                 2                             │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'text', 'embedding')   │
│   Multimodal dataclass   False                         │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   2                False             │
│   id          ('str',)       2                False             │
│   text        ('str',)       2                False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

Testing model...

In [13]:
images.match(text_docs, metric='cosine', limit=1)

## Detecting bias in finetuned `CLIP` results

## Sex (Disparate impact)

### Selection Rate (Positive results / N)

In [15]:
positive_males, positive_females = 0, 0

total_males, total_females = 0, 0

for i in range(len(images)):
   if images[i].tags['gender'] == 'male':
        total_males += 1
        if images[i].matches[0].text.split(' ')[-2] == 'male':
            positive_males += 1
   else:
        total_females += 1
        if images[i].matches[0].text.split(' ')[-2] == 'female':
            positive_females += 1
                
males_sr, females_sr = positive_males/ total_males, positive_females/ total_females

males_sr, females_sr

(0.9826472962066183, 0.997348652231551)

#### Disparate impact

In [16]:

# disparate impact ratio = underprivileged group SR / privileged group SR
disp_impact = females_sr / males_sr
disp_impact

1.0149609692935455

In [17]:
if disp_impact < 0.8:
    print('Disparate impact present in female group / male group')


## Sex (Equalized odds)

In [18]:
tp_males, tp_females, fn_males, fn_females, fp_males, fp_females = 0, 0, 0, 0, 0, 0
ages_of_fp=[]
for i in range(len(images)):
    if images[i].tags['gender'] == 'male':
        if images[i].matches[0].text.split(' ')[-2] == 'male':
            tp_males += 1
        else:
            fp_females += 1
            ages_of_fp.append(images[i].tags['age'])
            fn_males += 1 # False negative (wrong no male prediction, in this case, equal to female false positive)
    elif images[i].tags['gender'] == 'female':
        if images[i].matches[0].text.split(' ')[-2] == 'female':
            tp_females += 1
        else:
            fp_males += 1
            ages_of_fp.append(images[i].tags['age'])
            fn_females += 1
                
males_tpr, females_tpr = tp_males/ (tp_males + fn_males), tp_females/ (tp_females + fn_females)

males_fpr, females_fpr = fp_males/ (fp_males + fn_males), fp_females/ (fp_females + fn_females)


#### True Positive Rates

In [19]:
males_tpr, females_tpr

(0.9826472962066183, 0.997348652231551)

In [20]:
if abs(males_tpr - females_tpr) < 0.05:
    print('Equalized odds')
else:
    print('Not equalized odds')
    print(abs(males_tpr - females_tpr))

Equalized odds


#### False Positive Rates

In [68]:
males_fpr, females_fpr

(0.12244897959183673, 0.8775510204081632)

In [69]:
if abs(males_fpr - females_fpr) < 0.05:
    print('Equalized odds')
else:
    print('Not equalized odds')
    print(abs(males_fpr - females_fpr))

Not equalized odds
0.7551020408163265


In [71]:
wrong_class = []
for i in range(len(images)):
    if  images[i].tags['gender'] != images[i].matches[0].text.split(' ')[-2]:
        wrong_class.append(images[i].tags['filepath'])

wrong_class

['C:/Users/lachy/Pictures/utkface/utkface/10_0_0_20170110220251986.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/11_0_0_20170110220500946.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/15_0_0_20170104011743800.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/15_0_3_20170104225254497.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/16_0_0_20170110231736665.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/1_0_2_20161219200015132.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/1_0_3_20161219224956400.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/1_0_3_20161219225615528.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/1_0_3_20161220223100579.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/1_0_4_20161221192548675.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/1_1_0_20170109191027883.jpg.chip.jpg',
 'C:/Users/lachy/Pictures/utkface/utkface/21_0_2_20170116170638620.jpg.chip.jpg',
 'C:/Users/lachy/Pictu

Ruido en los datos:
Datos mal etiquetados:
'C:/Users/lachy/Pictures/utkface/utkface/55_0_0_20170116232628323.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/54_0_0_20170113210127075.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/50_1_3_20170117154120315.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/38_0_3_20170119202607101.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/36_1_3_20170117183408380.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/36_0_3_20170119180245724.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/36_0_0_20170117183240678.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/35_0_1_20170109001203061.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/33_0_0_20170117182541126.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/30_0_4_20170117202929576.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/27_0_2_20170116172745659.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/26_1_2_20170116182219997.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/26_0_4_20170117200550222.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/26_0_3_20170119181302068.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/26_0_0_20170113210127413.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/25_0_4_20170117151414412.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/24_0_2_20170116164749805.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/21_0_4_20161223214837442.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/21_0_4_20161223214835183.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/21_0_4_20161223214826657.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/21_0_4_20161223214831392.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/21_0_2_20170116170638620.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/1_0_3_20161220223100579.jpg.chip.jpg

Datos dificiles:
'C:/Users/lachy/Pictures/utkface/utkface/10_0_0_20170110220251986.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/55_0_0_20170116222002492.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/42_0_0_20170104184335702.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/2_1_4_20170103210746899.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/28_1_1_20170113150922335.jpg.chip.jpg'
'C:/Users/lachy/Pictures/utkface/utkface/1_1_0_20170109191027883.jpg.chip.jpg
'C:/Users/lachy/Pictures/utkface/utkface/1_0_4_20161221192548675.jpg.chip.jpg'
C:/Users/lachy/Pictures/utkface/utkface/1_0_3_20161219225615528.jpg.chip.jpg

## Race (Disparate impact)

Saving texts embeddings...(race)

In [40]:
text_docs = DocumentArray()
for item in race_tks:
    text_docs.append(Document(content=item))
text_docs[0].summary()
text_docs[1].summary()

finetuner.encode(model=clip_text_encoder, data=text_docs)

📄 Document: c01a09444f2784494cd1a0c52e9adb1a
╭───────────────────┬──────────────────────────────────────────────────────────╮
│ Attribute         │ Value                                                    │
├───────────────────┼──────────────────────────────────────────────────────────┤
│ text              │ A photo of a person of black race.                       │
╰───────────────────┴──────────────────────────────────────────────────────────╯

📄 Document: 22ca394f41f2455fbb32dc9f241470ae
╭───────────────────┬──────────────────────────────────────────────────────────╮
│ Attribute         │ Value                                                    │
├───────────────────┼──────────────────────────────────────────────────────────┤
│ text              │ A photo of a person of white race.                       │
╰───────────────────┴──────────────────────────────────────────────────────────╯

Output()

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Type                   DocumentArrayInMemory         │
│   Length                 5                             │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'text', 'embedding')   │
│   Multimodal dataclass   False                         │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   5                False             │
│   id          ('str',)       5                False             │
│   text        ('str',)       5                False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

Testing model...

In [41]:
images.match(text_docs, metric='cosine', limit=1)


### Selection Rate (Positive results / N)

In [42]:
positive_whites, positive_blacks, positive_asians, positive_indians = 0, 0, 0, 0

total_whites, total_blacks, total_asians, total_indians = 0, 0, 0, 0

for i in range(len(images)):
    if images[i].tags['race']== 'white':
        total_whites += 1
        if images[i].matches[0].text.split(' ')[-2] == 'white':
            positive_whites += 1
    elif images[i].tags['race'] == 'black':
        total_blacks += 1
        if images[i].matches[0].text.split(' ')[-2] == 'black':
            positive_blacks += 1
    elif images[i].tags['race'] == 'asian':
        total_asians += 1
        if images[i].matches[0].text.split(' ')[-2] == 'asian':
            positive_asians += 1
    elif images[i].tags['race'] == 'indian':
        total_indians += 1
        if images[i].matches[0].text.split(' ')[-2] == 'indian':
            positive_indians += 1
    else:
        continue
                
whites_sr, blacks_sr, asians_sr, indians_sr = positive_whites/ total_whites, positive_blacks/ total_blacks, positive_asians/ total_asians, positive_indians/ total_indians

whites_sr, blacks_sr, asians_sr, indians_sr

(0.9409429280397023,
 0.9900662251655629,
 0.9781659388646288,
 0.9459119496855346)

#### Disparate impact

In [24]:
# disparate impact ratio = underprivileged group SR / privileged group SR
disp_impact_b_w = blacks_sr/ whites_sr
disp_impact_b_a = asians_sr/ whites_sr
disp_impact_b_i = indians_sr/ whites_sr
disp_impact_b_w, disp_impact_b_a, disp_impact_b_i

(1.0505442041646178, 1.037916991475633, 1.0050274722545134)

In [76]:
if disp_impact_b_w < 0.8:
    print('Disparate impact present in black group / white group')
if disp_impact_b_a < 0.8:
    print('Disparate impact present in asian group / white group')
if disp_impact_b_i < 0.8:
    print('Disparate impact present in indian group / white group')

## Race (Equalized odds)


In [43]:
race_rates = {'white': {'tp': 0, 'fp': 0, 'fn': 0}, 'black': {'tp': 0, 'fp': 0, 'fn': 0}, 'asian': {'tp': 0, 'fp': 0, 'fn': 0}, 'indian': {'tp': 0, 'fp': 0, 'fn': 0}, 'other': {'tp': 0, 'fp': 0, 'fn': 0}}

for i in range(len(images)):
    race = images[i].tags['race']
    pred_race = images[i].matches[0].text.split(' ')[-2]

    if race == pred_race:
        race_rates[race]['tp'] += 1
    else:
        race_rates[race]['fn'] += 1
        race_rates[pred_race]['fp'] += 1

tpr = lambda tp, fn: tp/ (tp + fn)
fpr = lambda fp, fn: fp/ (fp + fn)

tpr_values = {'white': 0, 'black': 0, 'asian': 0, 'indian': 0, 'other': 0}
fpr_values = {'white': 0, 'black': 0, 'asian': 0, 'indian': 0, 'other': 0}
for race in race_rates.keys():
    rates = race_rates[race]
    tpr_values[race] = tpr(rates['tp'], rates['fn'])
    fpr_values[race] = fpr(rates['fp'], rates['fn'])

True Positive Rates

In [44]:
[tpr_value for tpr_value in tpr_values.values()]

[0.9409429280397023,
 0.9900662251655629,
 0.9781659388646288,
 0.9459119496855346,
 0.9171597633136095]

In [ ]:
from itertools import combinations

equalized_odds = True

for pair in combinations(tpr_values.keys(), 2):
    first_race = pair[0]
    second_race = pair[1]
    if first_race == 'other' or second_race == 'other':
        continue
    if abs(tpr_values[first_race] - tpr_values[second_race]) >= 0.05:
        equalized_odds = False
        print('Not equalized odds between ' + first_race + ' and ' + second_race)

if equalized_odds:
    print('Equalized odds')